In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from seqeval.metrics import accuracy_score, classification_report, f1_score

from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import layers, metrics, optimizers, losses, Model

from crf_layer import CRF
from preprocessing.contextNER import ContextNER

In [3]:
data = pd.read_csv('data/kaggle_ner.csv', sep=",", encoding="latin1").fillna(method='ffill')

In [4]:
data_ner = ContextNER(data)

In [5]:
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST  = train_test_split(data_ner.X_array,
                                                     data_ner.y_array, 
                                                     test_size=0.3)

# *Custom Model*

In [31]:
class NER_MODEL(Model):
    
    def __init__(self, 
                 configs_ner_params=None, 
                 dropout_rate=0.1,
                 use_crf=False,
                 hiden_units=128):
        
        super(NER_MODEL, self).__init__()

        self.configs_ner_params = configs_ner_params
        self.hiden_units = hiden_units
        self.use_crf = use_crf
        
        self.embedding = layers.Embedding(input_dim=configs_ner_params.num_words,
                                          output_dim=configs_ner_params.max_len,
                                          input_length=configs_ner_params.max_len)
        
        self.dropout = layers.TimeDistributed(layers.Dropout(dropout_rate))
            
        self.bilstm = layers.Bidirectional(layers.LSTM(units=hiden_units // 2,
                                                       return_sequences=True,
                                                       recurrent_dropout=0.1))
        
        self.dense_crf = layers.TimeDistributed(layers.Dense(units=configs_ner_params.num_tags, 
                                                             activation='relu'))

        self.classifier_crf = CRF(configs_ner_params.num_tags, 
                                  sparse_target=True)

        self.dense = layers.TimeDistributed(layers.Dense(units=self.hiden_units, 
                                                         activation='relu'))

        self.classifier_softmax = layers.TimeDistributed(layers.Dense(units=configs_ner_params.num_tags, 
                                                                      activation='softmax'))

    def call(self, inputs):

        X = self.embedding(inputs)
        X = self.dropout(X)
        X = self.bilstm(X)
        
        if self.use_crf:
            
            X = self.dense_crf(X)
            X = self.classifier_crf(X)
        else:
            
            X = self.dense(X)
            X = self.classifier_softmax(X)
            
        return X
        
    def model(self):
        
        """
            Implementação baseada na API Funcional, permite utilizar o método 'model.summary' 
            antes de executar o método 'model.fit'
            
            Retorna o modelo compilado com base no classificador CRF / SOFTMAX
            
            parâmetro: use_crf
        """
        
        X = layers.Input(shape=(self.configs_ner_params.max_len,))
        
        X = Model(inputs=[X], outputs=self.call(X))
        
        optm = optimizers.Adam(lr=0.001)

        if self.use_crf:
            X.compile(optimizer=optm,
                      loss=self.classifier_crf.loss,
                      metrics=[self.classifier_crf.accuracy])
        else:
            X.compile(optimizer=optm,
                      loss=losses.CategoricalCrossentropy(),
                      metrics=metrics.CategoricalAccuracy('accuracy'))
        return X

In [34]:
model = NER_MODEL(data_ner, use_crf=True).model()

In [35]:
model.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 105)]             0         
_________________________________________________________________
embedding_9 (Embedding)      (None, 105, 105)          3693900   
_________________________________________________________________
time_distributed_36 (TimeDis (None, 105, 105)          0         
_________________________________________________________________
bidirectional_9 (Bidirection (None, 105, 128)          87040     
_________________________________________________________________
time_distributed_37 (TimeDis (None, 105, 18)           2322      
_________________________________________________________________
crf_9 (CRF)                  (None, 105, 18)           324       
Total params: 3,783,586
Trainable params: 3,783,586
Non-trainable params: 0
_________________________________________________

In [ ]:
logging = TensorBoard(log_dir='./logs',
                      write_graph=True, 
                      write_images=True)

In [ ]:
model.fit(X_TRAIN, 
          Y_TRAIN, 
          validation_split=0.2, 
          batch_size=64, 
          epochs=5,
          callbacks=[logging])

In [ ]:
%load_ext tensorboard

%tensorboard --logdir logs/train

In [ ]:
preds = model.predict(X_TEST, verbose=1, batch_size=64)
                      
y_pred, y_true = \
np.argmax(preds, axis=-1), \
np.argmax(Y_TEST, -1)

pred_tag, true_tag = \
data_ner.parser2categorical(y_pred, y_true) 

In [ ]:
f1_score(pred_tag, true_tag)

In [ ]:
print(classification_report(pred_tag, true_tag))